## Survival Analysis: Based on Gradient Boosting Machine(GBM)

The tutorial give typical workflow of Gradient Boosting Desicion Tree-based survival analysis including data-preprocessing, model selection and traning&validation, uses R package `gbm`.

Formally, it can be listed by:
1. Data Preprocessing
  - convert variables
  - load training and test set
2. Model Selection
  - cross validation
  - tune parameters
3. Traning&Validation
  - train gbm model
  - measure CI on testset
  - survival rates on time of interest
  
The best suggestion about usage of `gbm` is official documentation named **"Generalized Boosted Models:A guide to the gbm package"**. Here, I summary some points related to usage of the package and SA, and disscussion of options to gbm that most users will need to change or tune.

### 1. Loss Function

`distribution` is corresponding to loss function and application. Here, the option `coxph` indicating SA should be selected.

### 2. Model Fitting

`shrinkage` and `n.trees` are mostly related to performance, so those should be tuned carefully.

As recommended by author of `gbm`, it is generally the case that for small shrinkage parameters, 0.001 for example.
>  My rule of thumb is to set shrinkage as small as possible while still being able to fit the model in a reasonable amount of time and storage. I usually aim for 3,000 to 10,000 iterations with shrinkage rates between 0.01 and 0.001.

### 3. The Optimal Number of Iterations

`gbm` offers three methods for estimating the optimal number of iterations after the gbm model has been fit, an independent test set (`test`), out-of-bag estimation (`OOB`), and v-fold cross validation (`cv`). The function `gbm.perf` computes the iteration estimate.

Among these methods for estimating `n.trees`, **V-Fold Cross Validation** is the best choice.
> My recommendation is to use 5- or 10-fold cross validation if you can afford the computing time. Otherwise you may choose among the other options, knowing that OOB is conservative.